In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
import os
from tqdm import tqdm_notebook
from skimage.transform import resize
from keras.preprocessing.image import load_img
itemgetter(*[0, 12])(a.split('/')[1].split(' ')[0])

Using TensorFlow backend.


In [64]:
path = '/media/danil/Data/Datasets/CelebA/Img'
path_to_img = '/media/danil/Data/Datasets/CelebA/Img/img_align_celeba'
BATCH_SIZE = 32

In [55]:
training = pd.read_csv(os.path.join(path,'list_attr_celeba.txt'), delimiter=" ")
training.Smiling = training.Smiling.map({1:1, -1:0})
training.head()

,ImageName,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,0,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,0,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,0,-1,-1,-1,-1,1,-1,-1,1


In [56]:
open_mouth_df = pd.read_csv(os.path.join(path,'AFLW.csv'))
open_mouth_df.columns = ['ImageName', 'open_mouth']
open_mouth_df.head()

,ImageName,open_mouth
0,055997.jpg,1
1,131827.jpg,1
2,052069.jpg,1
3,110305.jpg,1
4,098890.jpg,1


In [57]:
training = pd.merge(training, open_mouth_df, how='right', on='ImageName')

In [58]:
training.head()

,ImageName,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,open_mouth
0,000077.jpg,-1,-1,1,-1,-1,1,-1,-1,-1,...,1,-1,1,-1,-1,1,-1,-1,1,0
1,000090.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,1,-1,-1,-1,-1,-1,-1,1,0
2,000102.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,...,0,-1,-1,-1,-1,-1,-1,-1,1,0
3,000166.jpg,1,-1,-1,-1,-1,-1,-1,1,-1,...,0,-1,-1,-1,1,-1,-1,-1,-1,1
4,000307.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,1,-1,1


In [59]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
X_train_cv = skf.split(training.values, training['Smiling'].values)

index_train_dict = {}
index_test_dict = {}
for i in range(5):
    ind = next(X_train_cv)
    index_train_dict['split_{}'.format(i)] = ind[0]
    index_test_dict['split_{}'.format(i)] = ind[1]

In [60]:
from sklearn.utils import shuffle

In [143]:
def train_generator(train_id, training, batch_size = BATCH_SIZE):
    training = training[training.index.isin(index_train_dict['split_0'])]
    training = shuffle(training)
    out_rgb = []
    out_smile = []
    out_mouth = []
    while True:
        for path_img, smile, open_mouth in (training[['ImageName', 'Smiling', 'open_mouth']].values):
            out_rgb += [resize(imread(os.path.join(path_to_img, path_img)), (224,224,3))]
            out_smile += [smile]
            out_mouth += [open_mouth]
            if len(out_rgb)>=batch_size:
                yield np.stack(out_rgb, 0), [out_smile, out_mouth]
                out_rgb, out_smile, out_mouth=[], [], []

In [144]:
train_gen = train_generator(index_train_dict['split_0'], training)
val_gen = train_generator(index_test_dict['split_0'], training)

In [145]:
print(next(train_gen)[0].shape)
np.max(next(train_gen)[0])

/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(32, 224, 224, 3)


1.0

# Build Model

In [146]:
from keras.applications import NASNetMobile
model = NASNetMobile(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x1 = Dense(256, activation="elu", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x1 = BatchNormalization()(x1)
x1 = Dense(128, activation="elu", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x1)
x1 = BatchNormalization()(x1)

x2 = Dense(256, activation="elu", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x2 = BatchNormalization()(x2)
x2 = Dense(128, activation="elu", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x2)
x2 = BatchNormalization()(x2)

predictions1 = Dense(1, activation="sigmoid", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x1)
predictions2 = Dense(1, activation="sigmoid", kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x2)

from keras.models import Model
# creating the final model 
model_final = Model(input = model.input, output = [predictions1, predictions2])

/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`


In [147]:
from keras.optimizers import Adam
from sklearn.metrics import roc_curve, auc, roc_auc_score
from keras.models import Model

from keras import backend as K
import tensorflow as tf
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc
  
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model 
model_final.compile(loss = "binary_crossentropy", optimizer = Adam(lr=0.0001, decay=0.0001), metrics=['accuracy',auc,f1], loss_weights=[1., 1.])

In [148]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

weight_path='NashNetV2.h5'

checkpoint = ModelCheckpoint(weight_path, monitor='val_auc', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only=False)

reduceLROnPlat = ReduceLROnPlateau(monitor='f1', factor=0.1, patience=3, 
                                   verbose=1, mode='max', min_lr=0.0000001)

def exp_decay(epoch):
    initial_lrate = 0.0001
    k = 0.1
    lrate = initial_lrate * np.exp(-k*epoch)
    return lrate
lrate = LearningRateScheduler(exp_decay)

callbacks_list = [checkpoint, lrate]

In [149]:
loss_history = [model_final.fit_generator(train_gen, 
                             steps_per_epoch = 1600/BATCH_SIZE, 
                             epochs = 10,
                             validation_data = val_gen,
                             validation_steps = 1600/BATCH_SIZE,
                             callbacks = callbacks_list)]

Epoch 1/10


/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


50/50 [==============================] - 77s 2s/step - loss: 1.4644 - dense_11_loss: 0.6015 - dense_12_loss: 0.7272 - dense_11_acc: 0.7244 - dense_11_auc: 0.6714 - dense_11_f1: 0.7081 - dense_12_acc: 0.6925 - dense_12_auc: 0.6583 - dense_12_f1: 0.6857 - val_loss: 0.6342 - val_dense_11_loss: 0.2134 - val_dense_12_loss: 0.2852 - val_dense_11_acc: 0.9131 - val_dense_11_auc: 0.8719 - val_dense_11_f1: 0.8927 - val_dense_12_acc: 0.8850 - val_dense_12_auc: 0.8361 - val_dense_12_f1: 0.8672
Epoch 2/10


/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/keras/callbacks.py:432: RuntimeWarning: Can save best model only with val_auc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


50/50 [==============================] - 32s 630ms/step - loss: 0.3538 - dense_11_loss: 0.0973 - dense_12_loss: 0.1212 - dense_11_acc: 0.9769 - dense_11_auc: 0.9283 - dense_11_f1: 0.9744 - dense_12_acc: 0.9669 - dense_12_auc: 0.9050 - dense_12_f1: 0.9668 - val_loss: 0.2282 - val_dense_11_loss: 0.0464 - val_dense_12_loss: 0.0467 - val_dense_11_acc: 0.9938 - val_dense_11_auc: 0.9572 - val_dense_11_f1: 0.9930 - val_dense_12_acc: 0.9925 - val_dense_12_auc: 0.9407 - val_dense_12_f1: 0.9925
Epoch 3/10
50/50 [==============================] - 30s 603ms/step - loss: 0.1933 - dense_11_loss: 0.0231 - dense_12_loss: 0.0355 - dense_11_acc: 1.0000 - dense_11_auc: 0.9718 - dense_11_f1: 1.0000 - dense_12_acc: 0.9975 - dense_12_auc: 0.9595 - dense_12_f1: 0.9977 - val_loss: 0.1688 - val_dense_11_loss: 0.0160 - val_dense_12_loss: 0.0184 - val_dense_11_acc: 0.9994 - val_dense_11_auc: 0.9803 - val_dense_11_f1: 0.9993 - val_dense_12_acc: 1.0000 - val_dense_12_auc: 0.9710 - val_dense_12_f1: 1.0000
Epoch 4/1

# TEST

In [172]:
path_test = '/media/danil/Data/Datasets/example_data'
test_images = os.listdir(os.path.join(path_test,'images'))
test_images = [i for i in test_images if 'jpg' in i]

test_smile = os.listdir(os.path.join(path_test,'smile'))
test_smile = [i for i in test_smile if 'jpg' in i]

test_open_mouth = os.listdir(os.path.join(path_test,'open_mouth'))
test_open_mouth = [i for i in test_open_mouth if 'jpg' in i]

In [180]:
test_mouth = []
for i in test_images:
    if i in test_open_mouth:
        test_mouth += [1]
    else:
        test_mouth += [0]
sum(test_mouth)

38

In [179]:
test_smiles = []
for i in test_images:
    if i in test_smile:
        test_smiles += [1]
    else:
        test_smiles += [0]
sum(test_smiles)

23

In [181]:
test_df = pd.DataFrame()
test_df['image'] = test_images
test_df['smile'] = test_smiles
test_df['mouth'] = test_smiles

In [182]:
test_images = [resize(imread(os.path.join(path_test,'images', i)), (224,224,3)) for i in test_images]

/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/media/danil/Data/Kaggle/airbus-ship-detection/venv/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [197]:
from sklearn.metrics import roc_auc_score, f1_score

In [189]:
predict = model_final.predict(np.stack(test_images, 0))

In [201]:
score_smile = roc_auc_score(test_df.smile.values, predict[0])
score_mouth = roc_auc_score(test_df.mouth.values, predict[1])

f1_score_smile = f1_score(test_df.smile.values, np.round(predict[0]))
f1_score_mouth = f1_score(test_df.mouth.values, np.round(predict[1]))

In [202]:
print(f1_score_smile)
print(f1_score_mouth)

0.29545454545454547
0.3421052631578947
